# Data Transformation

In [0]:
-- CREATE SCHEMA TO STORE TRANSFORMED DATA

create schema if not exists transformed_data;

## Create Products Dimension

In [0]:
create table if not exists transformed_data.products_dim as
select
  product_id,
  product_name,
  category as product_category
from workspace.default.products;

## Create Orders Fact Table

In [0]:
-- Incrementally load orders_fact

MERGE INTO transformed_data.orders_fact AS target
USING (
  SELECT
    oi.item_id AS line_item_id,
    oi.order_id,
    oi.product_id,
    oi.quantity,
    o.order_date,
    o.customer_id,
    pr.price AS unit_price,
    ROUND(pr.price * oi.quantity, 2) AS total_price
  FROM workspace.default.order_items AS oi
  LEFT JOIN workspace.default.orders AS o 
    ON oi.order_id = o.order_id
  LEFT JOIN workspace.default.products AS pr 
    ON oi.product_id = pr.product_id
  WHERE o.order_date > (
    SELECT COALESCE(MAX(order_date), DATE('2000-01-01'))
    FROM transformed_data.orders_fact
  )
) AS source
ON target.line_item_id = source.line_item_id

WHEN MATCHED THEN UPDATE SET
  quantity = source.quantity,
  order_id = source.order_id,
  product_id = source.product_id,
  order_date = source.order_date,
  customer_id = source.customer_id,
  unit_price = source.unit_price,
  total_price = source.total_price

WHEN NOT MATCHED THEN INSERT (
  line_item_id, order_id, product_id, quantity, order_date, customer_id, unit_price, total_price
) VALUES (
  source.line_item_id, source.order_id, source.product_id, source.quantity,
  source.order_date, source.customer_id, source.unit_price, source.total_price
);


## Create Customer Dimension

In [0]:
create table if not exists transformed_data.customer_dim as
select
  customer_id,
  name as customer_name,
  email as email_address,
  city
from workspace.default.customers;

## Create Date Dimension

In [0]:
create table if not exists transformed_data.date_dim as
select
  order_date as date,
  year(order_date) as year,
  month(order_date) as month,
  day(order_date) as day,
  weekofyear(order_date) as week_of_year,
  dayofweek(order_date) as day_of_week,
  case 
    when dayofweek(order_date) in (1, 7) then 'Weekend'
    else 'Weekday'
  end as day_type,
  date_format(order_date, 'MMMM') as month_name,
  date_format(order_date, 'EEEE') as day_of_week_name
from workspace.default.orders
group by order_date;